<a href="https://colab.research.google.com/github/Stephane-Masamba/Team_4_CPT_ML-Classification/blob/Mikael/Mikael_Daniels_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import sklearn
import re
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import SGDClassifier, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from google.colab import drive

In [2]:
train = pd.read_csv('train (1).csv')
print(train.head())

   sentiment                                            message  tweetid
0          1  PolySciMajor EPA chief doesn't think carbon di...   625221
1          1  It's not like we lack evidence of anthropogeni...   126103
2          2  RT @RawStory: Researchers say we have three ye...   698562
3          1  #TodayinMaker# WIRED : 2016 was a pivotal year...   573736
4          1  RT @SoyNovioDeTodas: It's 2016, and a racist, ...   466954


In [3]:
test = pd.read_csv('test (1).csv')
print(test.head())

                                             message  tweetid
0  Europe will now be looking to China to make su...   169760
1  Combine this with the polling of staffers re c...    35326
2  The scary, unimpeachable evidence that climate...   224985
3  @Karoli @morgfair @OsborneInk @dailykos \nPuti...   476263
4  RT @FakeWillMoore: 'Female orgasms cause globa...   872928


In [4]:
sample = pd.read_csv('sample_submission.csv')
print(sample.head())

   tweetid  sentiment
0   169760          1
1    35326          1
2   224985          1
3   476263          1
4   872928          1


In [0]:
i = 0
for tweet in train['message']:
  tweet = tweet.lower()
  tweet = re.sub(r'http\S+', 'LINK', tweet)
  tweet = re.sub(r'[^\w\s]', '', tweet)
  tweet = tweet.lstrip()
  tweet = tweet.rstrip()
  tweet = tweet.replace('  ', ' ')
  train.loc[i, 'message'] = tweet
  i += 1

In [6]:
train

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesnt think carbon dio...,625221
1,1,its not like we lack evidence of anthropogenic...,126103
2,2,rt rawstory researchers say we have three year...,698562
3,1,todayinmaker wired 2016 was a pivotal year in ...,573736
4,1,rt soynoviodetodas its 2016 and a racist sexis...,466954
...,...,...,...
15814,1,rt ezlusztig they took down the material on gl...,22001
15815,2,rt washingtonpost how climate change could be ...,17856
15816,0,notiven rt nytimesworld what does trump actual...,384248
15817,-1,rt sara8smiles hey liberals the climate change...,819732


In [0]:
j = 0
for message in test['message']:
  message = message.lower()
  message = re.sub(r'http\S+', 'LINK', message)
  message = re.sub(r'[^\w\s]', '', message)
  message = message.lstrip()
  message = message.rstrip()
  message = message.replace('  ', ' ')
  test.loc[j, 'message'] = message
  j += 1

In [8]:
test

,message,tweetid
0,europe will now be looking to china to make su...,169760
1,combine this with the polling of staffers re c...,35326
2,the scary unimpeachable evidence that climate ...,224985
3,karoli morgfair osborneink dailykos \nputin go...,476263
4,rt fakewillmoore female orgasms cause global w...,872928
...,...,...
10541,rt brittanybohrer brb writing a poem about cli...,895714
10542,2016 the year climate change came home during ...,875167
10543,rt loop_vanuatu pacific countries positive abo...,78329
10544,rt xanria_00018 youre so hot you must be the c...,867455


In [9]:
train.isnull().sum()

sentiment    0
message      0
tweetid      0
dtype: int64

In [10]:
test.isnull().sum()

message    0
tweetid    0
dtype: int64

In [11]:
X_train = train['message']
X_train

0        polyscimajor epa chief doesnt think carbon dio...
1        its not like we lack evidence of anthropogenic...
2        rt rawstory researchers say we have three year...
3        todayinmaker wired 2016 was a pivotal year in ...
4        rt soynoviodetodas its 2016 and a racist sexis...
                               ...                        
15814    rt ezlusztig they took down the material on gl...
15815    rt washingtonpost how climate change could be ...
15816    notiven rt nytimesworld what does trump actual...
15817    rt sara8smiles hey liberals the climate change...
15818    rt chet_cannon kurteichenwalds climate change ...
Name: message, Length: 15819, dtype: object

In [12]:
y_train = train['sentiment']
y_train

0        1
1        1
2        2
3        1
4        1
        ..
15814    1
15815    2
15816    0
15817   -1
15818    0
Name: sentiment, Length: 15819, dtype: int64

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
tf_vect = TfidfVectorizer(ngram_range=(1, 2))

In [0]:
X_train = tf_vect.fit_transform(X_train)

In [0]:
X_test = test['message']

In [0]:
X_test = tf_vect.transform(X_test)

In [0]:
def accuracy(model):
     features = train['message']
     target = train['sentiment']

     train_scores = []
     test_scores = []

     tf_vect = TfidfVectorizer(ngram_range=(1, 2))

     folds = KFold(n_splits=5, shuffle=True)

     for train_index, test_index in folds.split(features):
          x_train, x_test = features.iloc[train_index], features.iloc[test_index]    
          y_train, y_test = target.iloc[train_index], target.iloc[test_index]

          x_train = tf_vect.fit_transform(x_train)
          x_test = tf_vect.transform(x_test)
     
          model.fit(x_train, y_train)
          train_predictions = model.predict(x_train)
          test_predictions = model.predict(x_test)

          train_score = accuracy_score(y_train, train_predictions)
          train_scores.append(train_score)

          test_score = accuracy_score(y_test, test_predictions)
          test_scores.append(test_score)

     avg_train_accuracy = np.mean(train_scores)
     avg_test_accuracy = np.mean(test_scores)

     return [avg_train_accuracy, avg_test_accuracy]

In [0]:
sv = SVC()

In [21]:
accuracy(sv)

[0.9808616118412343, 0.7296914625611477]

In [0]:
bernoulli = BernoulliNB()

In [23]:
accuracy(bernoulli)

[0.7547885911286645, 0.6341735370211753]

In [0]:
mnb = MultinomialNB()

In [25]:
accuracy(mnb)

[0.7016563013075288, 0.6336052764002873]

In [0]:
sgd = SGDClassifier()

In [27]:
accuracy(sgd)

[0.9598584080786365, 0.7519438769943079]

In [0]:
rand_forest = RandomForestClassifier()

In [29]:
accuracy(rand_forest)

[0.9992098122347366, 0.705354839637992]

In [0]:
knn = KNeighborsClassifier()

In [0]:
accuracy(knn)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['link', 'user_ref'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[0.7356028021184592, 0.6186872510175133]

In [0]:
grad_booster = GradientBoostingClassifier()

In [31]:
accuracy(grad_booster)

[0.7289177448688101, 0.6820282757372]

In [0]:
extra_trees = ExtraTreesClassifier()

In [33]:
accuracy(extra_trees)

[0.9992098122347368, 0.724887656863713]

In [0]:
bagging = BaggingClassifier()

In [35]:
accuracy(bagging)

[0.9796131246875033, 0.6381569170717202]

In [0]:
dec_tree = DecisionTreeClassifier()

In [37]:
accuracy(dec_tree)

[0.9991940044584947, 0.570895523581167]

In [0]:
linear_sv = LinearSVC()

In [39]:
accuracy(linear_sv)

[0.9977874732582727, 0.7606675318643625]

In [0]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [0]:
gs_clf = GridSearchCV(SVC(), tuned_parameters)

In [0]:
accuracy(gs_clf)

In [0]:
linear_svc = LinearSVC()

In [41]:
linear_svc.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
predictions = linear_svc.predict(X_test)

In [43]:
test['sentiment'] = predictions
test

,message,tweetid,sentiment
0,europe will now be looking to china to make su...,169760,1
1,combine this with the polling of staffers re c...,35326,1
2,the scary unimpeachable evidence that climate ...,224985,1
3,karoli morgfair osborneink dailykos \nputin go...,476263,1
4,rt fakewillmoore female orgasms cause global w...,872928,0
...,...,...,...
10541,rt brittanybohrer brb writing a poem about cli...,895714,1
10542,2016 the year climate change came home during ...,875167,1
10543,rt loop_vanuatu pacific countries positive abo...,78329,2
10544,rt xanria_00018 youre so hot you must be the c...,867455,0


In [44]:
submission = test[['tweetid', 'sentiment']]
submission

,tweetid,sentiment
0,169760,1
1,35326,1
2,224985,1
3,476263,1
4,872928,0
...,...,...
10541,895714,1
10542,875167,1
10543,78329,2
10544,867455,0
